# This notebook contains the experiments on Adult Census dataset with LionForests

In [1]:
import sys
cpath = !pwd
sys.path.append(cpath[0][:-18])
sys.path.append('C:\\Users\\iamollas\\Downloads\\LionForests Journal\\')
sys.path.append('C:\\Users\\iamollas\\Downloads\\LionForests Journal\\algorithms\\')

In [2]:
from lionforests import LionForests
from algorithms.simpleSurrogate import GlobalSurrogateTree, LocalSurrogateTree
from algorithms.DefragTrees.defragTrees import DefragModel
from CHIRPS.structures import data_container
import CHIRPS.routines as rt
import CHIRPS.structures as strcts

from scipy import sparse

from datasets.dataset import Dataset
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pandas as pd 
import numpy as np
np.seterr(invalid='ignore')
import warnings
warnings.filterwarnings("ignore")
import time

from sklearn.model_selection import train_test_split, LeaveOneOut
from sklearn.metrics import f1_score, precision_score

from anchor import anchor_tabular


Feature Engineering from:
https://github.com/pooja2512/Adult-Census-Income/blob/master/Adult%20Census%20Income.ipynb. So run and skip the next code block

In [3]:
adult = Dataset()
X, y, feature_names, class_names, categorical_features = adult.load_adult()
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1000, stratify=y, random_state=random_s)
#X = X_test
#y = y_test
y = np.array(y)

In [4]:
one_h_percent = int(min(10*len(X)/100,100))
print("Instances:",one_h_percent)
new_fn = []
for i in feature_names:
    new_fn.append(i.replace(' ','_'))
feature_names = new_fn
feature_names

Instances: 100


['workclass_Govt_employees',
 'workclass_Private',
 'workclass_Self_employed',
 'workclass_Without-pay',
 'education_Assoc-acdm',
 'education_Assoc-voc',
 'education_Bachelors',
 'education_Doctorate',
 'education_Elementary-school',
 'education_HS-grad',
 'education_Masters',
 'education_Preschool',
 'education_Prof-school',
 'education_Some-college',
 'marital-status_Married',
 'marital-status_Never-married',
 'marital-status_Separated',
 'marital-status_Widowed',
 'occupation_Adm-clerical',
 'occupation_Armed-Forces',
 'occupation_Craft-repair',
 'occupation_Exec-managerial',
 'occupation_Farming-fishing',
 'occupation_Handlers-cleaners',
 'occupation_Machine-op-inspct',
 'occupation_Other-service',
 'occupation_Priv-house-serv',
 'occupation_Prof-specialty',
 'occupation_Protective-serv',
 'occupation_Sales',
 'occupation_Tech-support',
 'occupation_Transport-moving',
 'race_Amer-Indian-Eskimo',
 'race_Asian-Pac-Islander',
 'race_Black',
 'race_Other',
 'race_White',
 'sex_Female',

In [5]:
def measure(X_train, X_test, y_train, y_test, feature_names, class_names):
    parameters = [{
        'max_depth': [5],
        'max_features': [0.75],
        'bootstrap': [True],
        'min_samples_leaf' : [2],
        'n_estimators': [500]
    }]
    scaler = MinMaxScaler(feature_range=(-1,1))
    lf = LionForests(None, False, scaler, feature_names, class_names)#, categorical_features)
    lf.fit(X_train, y_train, params=parameters)
    
    train = lf.utilizer.transform(X_train)
    test = lf.utilizer.transform(X_test)

    predictions = lf.model.predict(train)
    test_predictions = lf.model.predict(test)
    
    def techniques(model, train, y_train, predictions, test, feature_names, class_names, lf, task):

        #LionForests
        def lf_rule(instance):
            temp = lf.explain(instance, reduction=True, ar_algorithm='fpgrowth', cl_algorithm='SC')[-1]
            rule = {}
            for key,value in temp.items():
                rule[key] = [['<=',value[1]],['>',value[0]]]
            return rule
        print('    LF Ready')
        
        #BaselineTechnique ==============================================================================
        gt = GlobalSurrogateTree(train, predictions, feature_names, task)
        print('    GT Ready')
        lt = LocalSurrogateTree(train, predictions, feature_names, task, 150)
        print('    LT Ready')
        #DefragTechnique ================================================================================
        Kmax = 10
        splitter = DefragModel.parseSLtrees(model) # parse sklearn tree ensembles into the array of (feature index, threshold)
        mdl = DefragModel(modeltype=task, maxitr=10, qitr=0, tol=1e-6, restart=2, verbose=0)
        mdl.fit(train, predictions, splitter, Kmax, fittype='FAB', featurename=feature_names)
        def def_cov(instances):
            score, cover, coll = mdl.evaluate(instances[1:],lf.model.predict(instances[1:]))
            def_predictions = mdl.predict(instances[1:])
            rule = mdl.find_rule_length(instances[0])
            return rule, cover, def_predictions
        print('    Defrag Ready')

        #Anchors =======================================================================================
        explainer = anchor_tabular.AnchorTabularExplainer(class_names, feature_names, train)
        def anchors_method(instance):
            exp = explainer.explain_instance(instance, lf.model.predict, threshold=0.95)
            anchors_dict = {}
            for i in exp.names():
                terms = i.split(' ')
                if len(terms) == 3:
                    anchors_dict[terms[0]] = [[terms[1],float(terms[2])]]
                else:
                    anchors_dict[terms[2]] = [[terms[3],float(terms[4])],[terms[1],float(terms[0])]]
            return anchors_dict, 0
        print('    Anchors Ready')
        
        #CHIRPS =======================================================================================
        project_dir = 'C:\\Users\\iamollas\\Downloads\\LionForests Journal\\algorithms\\CHIRPS'
        temp_frame = pd.DataFrame(np.hstack((train,y_train.reshape(len(y_train),1))),columns=feature_names+['class'])
        temp_frame['class']=temp_frame['class'].astype(int)

        temp_frame = temp_frame.replace({"class": {1: 2}})
        temp_frame = temp_frame.replace({"class": {0: 1}})

        mydata = data_container(
                data = temp_frame, class_col = 'class', var_names = feature_names,
                project_dir = project_dir, save_dir = 'adult', random_state=123)
        meta_data = mydata.get_meta()
        f_walker = strcts.classification_trees_walker(forest=model, meta_data=meta_data)
        f_walker.forest_walk(instances = test, labels = model.predict(test), forest_walk_async = True)

        explanations = strcts.CHIRPS_container(f_walker.path_detail,
                                        forest=model,
                                        sample_instances=sparse.csr_matrix(train), # any representative sample can be used
                                        sample_labels=predictions,
                                        meta_data=meta_data)

        explanations.run_explanations(target_classes=model.predict(test), # we're explaining the prediction, not the true label!
                                explanation_async=False,
                                random_state=123,
                                which_trees='majority',
                                alpha_paths=0.0,
                                support_paths=0.1,
                                score_func=1,
                                precis_threshold=0.99,
                                disc_path_bins=4,
                                merging_bootstraps=20,
                                pruning_bootstraps=20,
                                delta=0.2,
                                weighting='kldiv')
        def chirps_method(idx):
            chirps_dict = {}
            for i in explanations.explainers[idx].pruned_rule:
                if i[1]:
                    chirps_dict[i[0]] = [['<=',float(i[2])]]
                else:
                    chirps_dict[i[0]] = [['>',float(i[2])]]
            return chirps_dict, 0
        print('    CHIRPS Ready')
       
        return {'gs':gt.rule,'ls':lt.rule,'an':anchors_method,'lf':lf_rule, 'df': def_cov, 'ch':chirps_method}
        #return {'gs':gt.rule,'ls':lt.rule,'an':anchors_method,'lf':lf_rule, 'df': def_cov}
    interpretation = techniques(lf.model, train, y_train, predictions, test, feature_names, class_names, lf, 'classification')
    def rule_cov(instance, feature_names, rule):
        covered = True
        for k in range(len(instance)):
            feature = feature_names[k]
            if feature in rule.keys():
                if len(rule[feature]) == 2:
                    if instance[k] > rule[feature][0][1]: #<=
                        covered = False
                    if instance[k] <= rule[feature][1][1]:
                        covered = False
                elif rule[feature][0][0] == '>':
                    if instance[k] <= rule[feature][0][1]:
                        covered = False
                else:
                     if instance[k] > rule[feature][0][1]: #<=
                        covered = False
        if covered:
            return 1
        else:
            return 0
        
    loo = LeaveOneOut()
    loo.get_n_splits(test)
    rule_generator = interpretation
    full_coverage = {'gs':0, 'ls':0, 'an':0, 'lf':0, 'df':0, 'ch': 0}
    rule_length = {'gs':0, 'ls':0, 'an':0, 'lf':0, 'df':0, 'ch': 0}
    f_precision = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch':[]}
    #full_coverage = {'gs':0, 'ls':0, 'an':0, 'lf':0, 'df':0}
    #rule_length = {'gs':0, 'ls':0, 'an':0, 'lf':0, 'df':0}
    #f_precision = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[]}

    for train_index, test_index in loo.split(test):
        x_train_temp, x_test_temp = test[train_index], test[test_index]
        y_train_temp, y_test_temp = test_predictions[train_index], test_predictions[test_index]
        x_train_temp_lf = lf.utilizer.inverse_transform(x_train_temp)
        x_test_temp_lf = lf.utilizer.inverse_transform(x_test_temp)
        for name, method in rule_generator.items():
            if name == 'ch':
                rule, op = method(test_index[0])
                coverage = 0
                precision = []
                co = 0
                for i in x_train_temp_lf:
                    res = rule_cov(i, feature_names, rule)
                    coverage = coverage + res
                    if res == 1:
                        precision.append([y_test_temp[0], y_train_temp[co]])
                    co = co + 1
                if len(precision) >= 1:
                    precision = np.array(precision)
                    f_precision[name].append(precision_score(precision[:,:1],precision[:,1:],average='micro'))
                full_coverage[name] = full_coverage[name] + coverage/len(x_train_temp)
            elif name == 'df':
                #FIX RULE LENGTH!
                rule, cover, predictions = method(np.concatenate((x_test_temp,x_train_temp)))
                f_precision[name].append(precision_score(predictions,y_train_temp,average='micro'))
                full_coverage[name] = full_coverage[name] + cover
            elif name == 'lf':
                rule = method(x_test_temp_lf[0])
                coverage = 0
                precision = []
                co = 0
                for i in x_train_temp_lf:
                    res = rule_cov(i, feature_names, rule)
                    coverage = coverage + res
                    if res == 1:
                        precision.append([y_test_temp[0], y_train_temp[co]])
                    co = co + 1
                if len(precision) >= 1:
                    precision = np.array(precision)
                    f_precision[name].append(precision_score(precision[:,:1],precision[:,1:],average='micro'))
                full_coverage[name] = full_coverage[name] + coverage/len(x_train_temp)
            else:
                rule, prediction = method(x_test_temp[0])
                coverage = 0
                precision = []
                co = 0 
                for i in x_train_temp:
                    if name == 'anchors':
                        res = rule_cov(i, feature_names, rule)
                    else:    
                        res = rule_cov(i, feature_names, rule)
                    coverage = coverage + res
                    if res == 1 and name=='anchors':
                        precision.append([y_test_temp[0], y_train_temp[co]])
                    elif res == 1:
                        precision.append([prediction, y_train_temp[co]])
                    co = co + 1
                if len(precision) >= 1:
                    precision = np.array(precision)
                    f_precision[name].append(precision_score(precision[:,:1],precision[:,1:],average='micro'))
                full_coverage[name] = full_coverage[name] + coverage/len(x_train_temp)
            rule_length[name] = rule_length[name] + len(rule)
    return rule_generator, full_coverage, rule_length, f_precision

In [7]:
def print_current(total_results):
    full_coverage = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch': []}
    rule_length = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch': []}
    f_precision = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch':[]}

    for i in total_results:
        for name, method in i[0].items():
            full_coverage[name].append(i[1][name]/one_h_percent)
            rule_length[name].append(i[2][name]/one_h_percent)
            if len(i[3][name]) >= 1:
                f_precision[name].append(np.array(i[3][name]).mean())
    for name, method in i[0].items():
        print(name,np.array(full_coverage[name]).mean(),',',np.array(full_coverage[name]).std(),
              np.array(rule_length[name]).mean(),',',np.array(rule_length[name]).std(),
              np.array(f_precision[name]).mean(),',',np.array(f_precision[name]).std())

In [8]:
total_results = []
rs = [7, 77, 777, 7777, 77777, 5, 55, 555, 5555, 55555]
for random_s in rs:
    print(random_s)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=one_h_percent, stratify=y, random_state=random_s)
    results = measure(X_train, X_test, y_train, y_test, feature_names, class_names)
    total_results.append(results)
    print_current([total_results[-1]])

7
    LF Ready
    GT Ready
    LT Ready
    Defrag Ready
    Anchors Ready
len self.path_detail
100
Working on CHIRPS for instance 0 of 100
as_chirps for batch_idx 0
start mining for batch_idx 0 with support = 0.1
found 57 patterns from 500 trees for batch_idx 0
start score sort for batch_idx 0 (57) patterns
start merge rule for batch_idx 0 (57) patterns
[('marital-status_Married', True, 0.0), ('capital-gain', True, -0.8583)]
0.9971833427221909 0.5080598029542698 0.12432482997746577 0.19493237991605783
merge complete for batch_idx 0 (57) patterns
start get explainer for batch_idx 0
as_chirps for batch_idx 1
start mining for batch_idx 1 with support = 0.1
found 75 patterns from 262 trees for batch_idx 1
start score sort for batch_idx 1 (75) patterns
start merge rule for batch_idx 1 (75) patterns
[('capital-gain', True, -0.898), ('education_Elementary-school', True, 0.0), ('education_HS-grad', True, 0.0), ('education_Some-college', False, -0.125), ('capital-loss', True, -0.1791)]
0.9997

found 43 patterns from 500 trees for batch_idx 16
start score sort for batch_idx 16 (43) patterns
start merge rule for batch_idx 16 (43) patterns
[('capital-gain', True, -0.898), ('education_HS-grad', False, -0.125), ('capital-loss', True, -0.1782)]
0.9998888827157064 0.39931218105169736 0.21524839849044763 0.22000486736432223
merge complete for batch_idx 16 (43) patterns
start get explainer for batch_idx 16
as_chirps for batch_idx 17
start mining for batch_idx 17 with support = 0.1
found 55 patterns from 437 trees for batch_idx 17
start score sort for batch_idx 17 (55) patterns
start merge rule for batch_idx 17 (55) patterns
[('education_Bachelors', False, -0.125), ('marital-status_Married', False, -0.125)]
0.9773584905660377 0.0879412065365101 0.07514632395938058 0.05227397260273973
merge complete for batch_idx 17 (55) patterns
start get explainer for batch_idx 17
as_chirps for batch_idx 18
start mining for batch_idx 18 with support = 0.1
found 58 patterns from 500 trees for batch_id

found 43 patterns from 500 trees for batch_idx 33
start score sort for batch_idx 33 (43) patterns
start merge rule for batch_idx 33 (43) patterns
[('capital-gain', True, -0.898), ('education_HS-grad', False, -0.125), ('capital-loss', True, -0.17822)]
0.9998888827157064 0.39931218105169736 0.21515449573534534 0.21981499513145092
merge complete for batch_idx 33 (43) patterns
start get explainer for batch_idx 33
as_chirps for batch_idx 34
start mining for batch_idx 34 with support = 0.1
found 61 patterns from 500 trees for batch_idx 34
start score sort for batch_idx 34 (61) patterns
start merge rule for batch_idx 34 (61) patterns
[('marital-status_Married', True, 0.0), ('capital-gain', True, -0.85826)]
0.9971833427221909 0.5080598029542698 0.21651625330782498 0.33276133276133285
merge complete for batch_idx 34 (61) patterns
start get explainer for batch_idx 34
Working on CHIRPS for instance 35 of 100
as_chirps for batch_idx 35
start mining for batch_idx 35 with support = 0.1
found 57 patt

found 50 patterns from 500 trees for batch_idx 50
start score sort for batch_idx 50 (50) patterns
start merge rule for batch_idx 50 (50) patterns
[('marital-status_Married', True, 0.0), ('capital-gain', True, -0.85823)]
0.9971833427221909 0.5080598029542698 0.1370327825856144 0.22058294350485796
merge complete for batch_idx 50 (50) patterns
start get explainer for batch_idx 50
as_chirps for batch_idx 51
start mining for batch_idx 51 with support = 0.1
found 61 patterns from 500 trees for batch_idx 51
start score sort for batch_idx 51 (61) patterns
start merge rule for batch_idx 51 (61) patterns
[('marital-status_Married', True, 0.0), ('capital-gain', True, -0.85826)]
0.9971833427221909 0.5080598029542698 0.2165163321970996 0.33276133276133285
merge complete for batch_idx 51 (61) patterns
start get explainer for batch_idx 51
as_chirps for batch_idx 52
start mining for batch_idx 52 with support = 0.1
reduced 3 patterns out of 56 by numeric redundancy
found 53 patterns from 479 trees for 

reduced 3 patterns out of 44 by numeric redundancy
found 41 patterns from 500 trees for batch_idx 67
start score sort for batch_idx 67 (41) patterns
start merge rule for batch_idx 67 (41) patterns
[('capital-gain', False, -0.898)]
0.953887884267631 0.048901739769971246 0.12893945323487177 0.2357360042926131
merge complete for batch_idx 67 (41) patterns
start get explainer for batch_idx 67
as_chirps for batch_idx 68
start mining for batch_idx 68 with support = 0.1
found 47 patterns from 500 trees for batch_idx 68
start score sort for batch_idx 68 (47) patterns
start merge rule for batch_idx 68 (47) patterns
[('capital-gain', True, -0.858), ('marital-status_Married', True, 0.0)]
0.9971833427221909 0.5080598029542698 0.11756661426853633 0.21407942238267147
merge complete for batch_idx 68 (47) patterns
start get explainer for batch_idx 68
as_chirps for batch_idx 69
start mining for batch_idx 69 with support = 0.1
found 60 patterns from 500 trees for batch_idx 69
start score sort for batch_

...

start merge rule for batch_idx 26 (39) patterns
[('capital-gain', False, -0.89801)]
0.9538461538461539 0.04885751367012688 0.141275532531573 0.2613704071499503
merge complete for batch_idx 26 (39) patterns
start get explainer for batch_idx 26
as_chirps for batch_idx 27
start mining for batch_idx 27 with support = 0.1
found 38 patterns from 500 trees for batch_idx 27
start score sort for batch_idx 27 (38) patterns
start merge rule for batch_idx 27 (38) patterns
[('capital-gain', True, -0.89801), ('capital-loss', True, -0.17864), ('education_HS-grad', False, -0.125)]
0.9998888580161156 0.3992234302196583 0.1547012372620962 0.17829457364341092
merge complete for batch_idx 27 (38) patterns
start get explainer for batch_idx 27
as_chirps for batch_idx 28
start mining for batch_idx 28 with support = 0.1
found 54 patterns from 500 trees for batch_idx 28
start score sort for batch_idx 28 (54) patterns
start merge rule for batch_idx 28 (54) patterns
[('marital-status_Married', True, 0.0), ('capi

[('education_Bachelors', False, -0.125), ('marital-status_Married', False, -0.125)]
0.9831361691417065 0.0879519752027526 0.06495072369165446 0.04844149207971388
merge complete for batch_idx 43 (67) patterns
start get explainer for batch_idx 43
as_chirps for batch_idx 44
start mining for batch_idx 44 with support = 0.1
found 64 patterns from 390 trees for batch_idx 44
start score sort for batch_idx 44 (64) patterns
start merge rule for batch_idx 44 (64) patterns
[('education_Elementary-school', True, 0.0), ('education_HS-grad', True, 0.0), ('marital-status_Married', False, -0.125), ('education_Some-college', True, 0.0), ('occupation_Prof-specialty', False, -0.125)]
0.9898534385569335 0.058957090417770155 0.3134986657148382 0.26865988125530105
merge complete for batch_idx 44 (64) patterns
start get explainer for batch_idx 44
Working on CHIRPS for instance 45 of 100
as_chirps for batch_idx 45
start mining for batch_idx 45 with support = 0.1
found 69 patterns from 273 trees for batch_idx 

start merge rule for batch_idx 60 (68) patterns
[('education_Elementary-school', True, 0.0), ('education_HS-grad', True, 0.0), ('marital-status_Married', False, -0.125), ('education_Some-college', True, 0.0), ('occupation_Prof-specialty', False, -0.125)]
0.9898534385569335 0.058957090417770155 0.3104080503674259 0.2627789046653143
merge complete for batch_idx 60 (68) patterns
start get explainer for batch_idx 60
as_chirps for batch_idx 61
start mining for batch_idx 61 with support = 0.1
found 71 patterns from 234 trees for batch_idx 61
start score sort for batch_idx 61 (71) patterns
start merge rule for batch_idx 61 (71) patterns
[('capital-gain', True, -0.89809), ('capital-loss', True, -0.17863)]
0.8777504342790967 0.35275078300305457 0.07559084429569352 0.07856989937914788
merge complete for batch_idx 61 (71) patterns
start get explainer for batch_idx 61
as_chirps for batch_idx 62
start mining for batch_idx 62 with support = 0.1
found 59 patterns from 500 trees for batch_idx 62
start

found 64 patterns from 407 trees for batch_idx 77
start score sort for batch_idx 77 (64) patterns
start merge rule for batch_idx 77 (64) patterns
[('education_Elementary-school', True, 0.0), ('education_HS-grad', True, 0.0), ('marital-status_Married', False, -0.125), ('education_Some-college', True, 0.0), ('occupation_Prof-specialty', False, -0.125)]
0.9898534385569335 0.058957090417770155 0.3154743010736811 0.270396999062207
merge complete for batch_idx 77 (64) patterns
start get explainer for batch_idx 77
as_chirps for batch_idx 78
start mining for batch_idx 78 with support = 0.1
reduced 2 patterns out of 50 by numeric redundancy
found 48 patterns from 500 trees for batch_idx 78
start score sort for batch_idx 78 (48) patterns
start merge rule for batch_idx 78 (48) patterns
[('capital-gain', False, -0.898)]
0.9538461538461539 0.04885751367012688 0.11899741224484009 0.216954732510288
merge complete for batch_idx 78 (48) patterns
start get explainer for batch_idx 78
as_chirps for batch_

start merge rule for batch_idx 94 (67) patterns
[('education_Bachelors', False, -0.125), ('marital-status_Married', False, -0.125)]
0.9831361691417065 0.0879519752027526 0.06480620898950397 0.04829778506972931
merge complete for batch_idx 94 (67) patterns
start get explainer for batch_idx 94
Working on CHIRPS for instance 95 of 100
as_chirps for batch_idx 95
start mining for batch_idx 95 with support = 0.1
found 67 patterns from 444 trees for batch_idx 95
start score sort for batch_idx 95 (67) patterns
start merge rule for batch_idx 95 (67) patterns
[('education_HS-grad', True, 0.0), ('marital-status_Married', False, -0.125), ('education_Elementary-school', True, 0.0), ('education_Some-college', True, 0.0)]
0.8397221188438159 0.1725511498922677 0.3076349217997406 0.2655783396376666
merge complete for batch_idx 95 (67) patterns
start get explainer for batch_idx 95
as_chirps for batch_idx 96
start mining for batch_idx 96 with support = 0.1
found 74 patterns from 180 trees for batch_idx 9

In [9]:
print_current(total_results)

gs 0.3179393939393939 , 0.029810105391129733 4.66 , 0.16376812876747432 1.0 , 0.0
ls 0.747222222222222 , 0.02893770157408097 1.236 , 0.07391887445030533 0.7932643737201077 , 0.0424885972531263
an 0.31800000000000006 , 0.027145111077365572 2.815 , 0.4477778467052608 0.8231293745876037 , 0.018416126017883186
lf 0.0 , 0.0 8.894 , 0.40348977682216425 nan , nan
df 1.0 , 0.0 6.114 , 2.313552247086718 0.8639999999999997 , 0.03852272056851645
ch 0.14853535353535352 , 0.023678467636424932 2.588 , 0.11187493016757595 0.27998705427344267 , 0.04474572348384671
